# 马尔科夫决策过程与动态规划
在本节内容中，我们将从马尔可夫过程出发，引出马尔可夫决策过程，并介绍如何通过动态规划得到最优策略。

**本notebook包含7个练习(Exercise 1\~7)和5个代码填空(Programming 1\~5)。**

# 马尔可夫过程（Markov Process）
### 随机过程（Stochastic Process）

随机过程是概率论的“动力学”部分。概率论的研究对象是静态的随机现象，而随机过程的研究对象是随时间演变的随机现象。例如，天气随时间的变化，城市交通随时间的变化。随机过程中，随机现象在某时刻$t$的取值被称为状态$S_t$，所有可能的状态组成状态集合$\mathcal{S}$。于是，随机现象研究的便是状态的变化过程。在某时刻$t$的状态$S_t$通常取决于$t$时刻之前的状态。我们将已知历史信息$（S_1,...,S_t）$时下一个时刻状态为$S_{t+1}$的概率表示成$P(S_{t+1}|S_1,...,S_t)$。

### 马尔可夫性质（Markov Property）
一个随机过程被称为具有马尔可夫性质，当且仅当某时刻的状态只却决于上一时刻的状态，用公式表示为$P(S_{t+1}|S_t) = P(S_{t+1}|S_1,...,S_t)$。即下一个状态只取决于当前状态，而不会受到过去状态的影响。需要明确的是，马尔可夫性并不代表这个随机过程就和历史完全没有关系。因为虽然$t+1$时刻的状态只与$t$时刻的状态有关，但是$t$时刻的状态其实包含了$t-1$时刻的状态的信息，通过这种链式的关系，历史的信息被传递到了现在。马尔可夫性可以大大简化运算，因为只要当前状态可知，所有的历史信息都不再需要了，利用当前状态信息就可以决定未来。

### 马尔可夫过程（Markov Process）
马尔可夫过程指具有马尔可夫性质的随机过程，也被称为马尔可夫链（Markov Chain）。我们通常用元组$<\mathcal{S},\mathcal{P}>$描述一个马尔可夫过程，其中$\mathcal{S}$是有限数量的状态集合，$\mathcal{P}$是状态转移矩阵（State Transition Matrix）。我们假设一共有$n$个状态，此时$\mathcal{S}=\{s_1,s_2,...,s_n\}$。状态转移矩阵$\mathcal{P}$定义了所有状态对之间的转移概率，即
$$
\mathcal{P} = \left[ \begin{matrix} p(s_1|s_1) & ... & p(s_n|s_1) \\... \\ p(s_1|s_n)  & ... & p(s_n|s_n) \end{matrix} \right]
$$
矩阵$\mathcal{P}$中第$i$行第$j$列元素$p(s_j|s_i) = P(S_{t+1}=s_j|S_t=s_i)$表示从状态$s_i$转移到状态$s_j$的概率，我们称$p(s'|s)$为状态转移函数。从某个状态出发，到达其他状态的概率和必须为1，即状态转移矩阵$\mathcal{P}$的每一行的和为1。

**下图是一个具有6个状态的马尔科夫过程的简单例子**。其中每个绿色圆圈表示一个状态，每个状态都有一定概率（包括概率为零）转移到其他状态，其中$s_6$通常被称为终止状态（terminal state），因为它不会再转移到其他状态，可以理解为它永远以概率1转移到自己。状态之间的虚线箭头表示状态的转移，箭头旁的数字表示该状态转移发生的概率。从每个状态出发转移到其他状态的概率总和为1。比如说，$s_1$有90%概率保持不变，有10%概率转移到$s_2$，而在$s_2$又有50%概率回到$s_1$，有50%概率转移到$s_3$。

![Image Name](https://cdn.kesci.com/upload/image/qgcdfsic36.PNG?imageView2/0/w/360/h/360)
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; 图：马尔可夫过程的一个简单例子



我们可以写出这个马尔可夫过程的状态转移矩阵$\mathcal{P}= \left [\begin{matrix} 0.9 & 0.1 & 0 & 0 & 0 & 0\\ 0.5& 0 &0.5 &0 &0 &0 \\0& 0& 0&0.6&0 &0.4 \\ 0& 0& 0& 0& 0.3&0.7 \\0 &0.2&0.3&0.5 &0&0\\ 0&0 &0 &0 &0 &1\end{matrix} \right]$

给定一个马尔可夫过程，我们就可以从某个状态出发，根据它的状态转移矩阵生成一个状态**序列（episode）**，这个步骤也被叫做**采样（sample）**。例如：从$s_1$出发，可以生成序列$s_1-s_2-s_3-s_6$ 或序列 $s_1-s_1-s_2-s_3-s_4-s_5-s_3-s_6$等等。生成这些序列的概率和状态转移矩阵有关。


# 马尔可夫奖励过程（Markov Reward Process）
在马尔可夫过程的基础上加入奖励函数 $\mathcal{r}$和折扣因子 $\mathcal{\gamma}$，我们就可以得到马尔可夫奖励过程。于是，一个马尔可夫奖励过程由$<\mathcal{S},\mathcal{P},\mathcal{r},\gamma>$构成，其中
* $\mathcal{S}$是有限状态的集合
* $\mathcal{P}$是状态转移矩阵
* $r$是奖励函数，某个状态$s$的奖励 $r(s)$指转移到该状态时可以获得的奖励
* $\gamma$是折扣因子（Discount Factor），$\gamma$的取值范围为$[0,1)$。引入折扣因子的理由为远期利益具有一定不确定性，有时我们更希望能够尽快获得一些奖励，所以我们需要对远期利益打一些折扣。接近1的$\gamma$更关注长期的累计奖励，接近0的$\gamma$更考虑短期奖励。

### 回报（Return）
在一个马尔可夫奖励过程中，从某一状态$S_t$开始直到终止状态时所有奖励的衰减之和称为回报$G_t$，用$R_t$表示在时刻$t$获得的奖励。

下图中，我们继续沿用之前马尔可夫链的例子，然后在此基础上添加了奖励函数，构建成一个马尔可夫奖励过程。比如，进入状态$s_2$可以得到奖励-2，表明我们不希望进入$s_2$，进入$s_4$可以获得最高的奖励10，但是进入$s_6$之后奖励为零，并且此时序列也终止了。


![Image Name](https://cdn.kesci.com/upload/image/qgcd3cjbzb.PNG?imageView2/0/w/360/h/360)
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; 图：马尔可夫奖励过程的一个简单例子

如果我们选取$s_1$为起始状态，设置$\gamma$=0.5，采样到一条状态序列为$s_1-s_2-s_3-s_6$，就可以计算$s_1$的回报$G_1$，具体计算过程请写在**Exercise 1**中。

接下来我们将图中马尔可夫奖励过程用代码表示，并且定义计算回报的函数。

**Exercise 1 请写出$G_t$的公式，并计算上文例子中$G_1$的值**\
$G_t=\sum_{i=0}^{\infty} \gamma ^i \mathcal{r}_{t+i}$\
$G_1$  is  -2.5

In [1]:
# 导入库
import numpy as np         
import random
# 定义状态转移概率矩阵P
P = [
    [0.9, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.6, 0.0, 0.4],
    [0.0, 0.0, 0.0, 0.0, 0.3, 0.7],
    [0.0, 0.2, 0.3, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
]                           
P = np.array(P)   
# 定义奖励函数
rewards = [-1, -2, -2, 10, 1, 0]  
# 定义折扣因子
gamma = 0.5                 
# 给定一条序列，计算从某个索引开始到序列最后得到的回报
def compute_return(start_index, chain, gamma):   
    rt = 0
    ########################################
    ## Programming 1：计算$G_t$
    ########################################
    chain.reverse()
    for state in chain:
        rt = rewards[state-1] + rt*gamma
    return rt

In [2]:
# 一个状态序列，可以选择其他序列
chain = [1, 2, 3, 6]    
start_index = 0
G = compute_return(start_index, chain, gamma)
print("根据本序列计算得到回报为：%s，正确结果为-2.5" % G)

根据本序列计算得到回报为：-2.5，正确结果为-2.5


### 价值函数（Value Function）
在马尔可夫奖励过程中，一个状态的期望回报被称为这个状态的价值（value）。所有状态的价值就组成了价值函数，价值函数的输入为某个状态，输出为这个状态的价值。我们将价值函数写成$V(s)=\mathbb{E}[G_t|S_t=s]$。我们将其展开为
$$
\begin{align}
V(s)&= \mathbb{E}[G_t|S_t=s]     \nonumber\\
  &= \mathbb{E}[R_{t}+\gamma R_{t+1}+\gamma^2R_{t+2}+...|S_t=s]  \nonumber \\
	&= \mathbb{E}[R_{t}+\gamma (R_{t+1}+\gamma R_{t+2}+...)|S_t=s]  \nonumber  \\
	&= \mathbb{E}[R_{t}+\gamma G_{t+1}|S_t=s]    \nonumber \\
	&=\mathbb{E}[R_{t}+\gamma V(S_{t+1})|S_t=s]  \nonumber
\end{align}
$$
上式的最后一个等号中，一方面，即时奖励的期望就等于即时奖励，即$\mathbb{E}[R_{t}|S_t=s]=r(s)$，另一方面，式子中剩余部分$\mathbb{E}[\gamma V(S_{t+1})|S_t=s]$可以根据从状态$s$出发的转移概率得到，即我们可以得到
$$ 
V(s)=r(s) + \gamma \sum_{s' \in S}p(s'|s)V(s') 
$$
上式就是马尔可夫奖励过程中非常有名的**贝尔曼方程（Bellman Equation）**。
上式的贝尔曼方程对每一个状态都成立。若一个马尔可夫奖励过程一共有$n$个状态，即$\mathcal{S}=\{ s_1,s_2,...,s_n  \}$，我们将所有状态的价值表示成一个列向量$\mathcal{V}=[V(s_1), V(s_2), ..., V(s_n)]^T$，同理奖励函数写成一个列向量$\mathcal{R}=[r(s_1),r(s_2),...,r(s_n) ]^T$。于是我们可以将贝尔曼方程写成矩阵的形式
$$
\mathcal{V}=\mathcal{R}+\gamma \mathcal{P} \mathcal{V}
$$
$$
\left [\begin{matrix} V(s_1)\\V(s_2) \\...\\V(s_n) \end{matrix} \right] = \left [\begin{matrix} r(s_1)\\r(s_2)\\...\\ r(s_n) \end{matrix} \right] + \gamma \left [\begin{matrix} p(s_1 |s_1)& p(s_2 |s_1)& ...& p(s_n |s_1)\\p(s_1 |s_2)& p(s_2 |s_2)& ...& p(s_n |s_2) \\...\\p(s_1 |s_n)& p(s_2 |s_n) & ... & p(s_n |s_n) \end{matrix} \right] \left [\begin{matrix} V(s_1)\\V(s_2) \\...\\V(s_n) \end{matrix} \right] 
$$
于是我们可以直接根据矩阵运算求解得到以下解析解：
$$
\begin{align}
\mathcal{V} &=\mathcal{R}+\gamma \mathcal{PV}  \nonumber\\
(I - \gamma \mathcal{P})\mathcal{V} &= \mathcal{R} \nonumber\\
\mathcal{V} &=(I-\gamma \mathcal{P})^{-1} \mathcal{R} \nonumber
\end{align}
$$
但实际上，解析解的计算复杂度是$O(n^3),n$是状态个数，所以这种方法只适用很小的马尔可夫奖励过程。求解较大规模的马尔可夫奖励过程中的价值函数，我们可以使用动态规划（Dynamic Programming），蒙特卡洛法（Monte-Carlo method），时序差分法（Temporal Difference），这些方法我们将会在之后学到。

我们接下来将求解价值函数的解析解方法用代码写出来，并据此计算上图MRP中所有状态的价值。

In [3]:
# 利用贝尔曼方程矩阵形式计算解析解,states_num是MRP的状态数
def compute(P, rewards, gamma, states_num): 
    ##rewards列向量的形状为（6，1）
    rewards = np.array(rewards).reshape((-1,1))        
    value = np.dot(np.linalg.inv(np.eye(states_num,states_num) - gamma * P), rewards)
    return value

states_num = 6
V = compute(P, rewards, gamma, states_num)
print("MRP中每个状态价值分别为\n", V)
# 可以修改P和rewards，自行实验其他情况

MRP中每个状态价值分别为
 [[-2.01950168]
 [-2.21451846]
 [ 1.16142785]
 [10.53809283]
 [ 3.58728554]
 [ 0.        ]]


根据以上代码我们求解得到各个状态的价值函数$V(s)$，具体如下
$$
\left [\begin{matrix} V(s_1)\\ V(s_2) \\V(s_3) \\ V(s_4)\\V(s_5)\\V(s_6) \end{matrix} \right] = \left [\begin{matrix} -2.02\\-2.21 \\1.16\\ 10.54 \\3.59 \\ 0 \end{matrix} \right]
$$
我们现在用贝尔曼方程来进行简单验证。对于状态$s_4$来说，当$\gamma=0.5$时，我们有
$$
V(s_4)=r(s_4) + \gamma \sum_{s' \in \mathcal{S}}P(s'|s_4)V(s')\\
10.54 = 10+ 0.5\times (0.7\times 0+0.3\times 3.59)
$$
我们发现左右两边是相等的，说明我们求解得到的价值函数是满足状态为$s_4$时的贝尔曼方程。同学们可以自行验证其他状态时候的贝尔曼方程是否也成立。若对于所有状态都成立，就可以说明我们求解得到的价值函数是正确的。马尔可夫奖励过程中的价值函数也可以通过蒙特卡洛的方法估计得到，我们将在本节课程接下来的马尔可夫决策过程中进行介绍该方法。

# 马尔可夫决策过程（Markov Decision Process）
在马尔可夫奖励过程MRP的基础上加入动作，就得到了马尔可夫决策过程MDP。马尔科夫决策过程由元组$\langle\mathcal{S},\mathcal{A},\mathcal{P},\mathcal{r},\mathcal{\gamma}\rangle$构成，其中：
* $\mathcal{S}$是状态的集合
* $\mathcal{A}$是动作的集合
* $\mathcal{\gamma}$是折扣因子
* $\mathcal{r}(s,a)$是奖励函数，此时奖励同时取决于状态$s$和动作$a$
* $p(s'|s,a)$是状态转移函数，表示在状态$s$执行动作$a$之后到达状态$s'$的概率

我们发现MDP与MRP非常相像，主要区别为MDP中的状态转移函数和奖励函数都比MRP多了动作$a$作为自变量。注意在上面MDP的定义中，我们不再使用类似MRP定义中的状态转移矩阵方式，而是直接表示成了状态转移函数。一是此时状态转移与动作也有关，变成了一个三维数组，而不再是一个矩阵（二维数组）；二是状态转移函数更具有一般意义，例如当状态集合不是有限的时候，就无法用数组表示，但我们仍可以用状态转移函数表示。我们在之后的课程学习中会遇到连续状态的MDP环境，那时状态集合都不是有限的。现在我们主要关注于离散状态的MDP环境，此时状态集合是有限的。

不同于马尔可夫奖励过程，在马尔可夫决策过程中，通常存在一个智能体（agent）来执行动作。举个例子，如果一艘小船在大海中随着水流自由飘荡的过程就是一个马尔可夫奖励过程，它如果凭借运气漂到了一个目的地就能获得比较大的奖励；如果有个水手在控制着这条船往哪个方向前进，就可以主动选择前往目的地获得比较大的奖励。因为这是一个与时间相关的不断进行的过程，在智能体和环境MDP之间存在一个不断交互的过程。一般而言，它们之间的交互是如图循环过程：智能体根据当前状态$S_t$选择动作$A_t$；对于状态$S_t$和动作$A_t$，MDP根据奖励函数和状态转移函数得到$S_{t+1}$和$R_t$并反馈给智能体。智能体的目标是最大化得到的累计奖励。智能体根据当前状态从动作的集合$\mathcal{A}$中选择一个动作的函数，被称为策略。


![Image Name](https://cdn.kesci.com/upload/image/qgfl65ncrh.PNG?imageView2/0/w/360/h/360)

&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; 图：智能体与环境MDP的交互示意图

### 策略（Policy）
智能体的策略通常用字母$\pi$表示。策略$\pi(a|s)= p(A_t = a|S_t = s)$是一个函数，表示在输入状态$s$情况下采取动作$a$的概率。当一个策略是**确定性策略**时，它在每个状态时只输出一个确定性的动作，即只有该动作的概率为1，其他动作的概率为0；当一个策略是**随机性**策略时，它在每个状态时输出的是关于动作的分布，然后根据该分布进行采样就可以得到一个动作。在MDP中，由于马尔可夫性质的存在，我们的策略只需要与当前状态有关，不需要考虑历史状态。回顾一下在MRP中的价值函数，在MDP中也同样可以定义类似的价值函数。但此时的价值函数与策略有关，这意为着对于两个不同的策略来说，它们在同一个状态上的价值也是不同的。这很好理解，因为不同的策略会采取不同的动作，从而之后会遇到不同的状态以及获得不同的奖励，于是它们的累计奖励的期望也就不同，即状态价值不同。
### 状态价值函数（State-value Function）
我们用$V_{\pi}(s)$表示在MDP中基于策略$\pi$的状态价值函数，定义为从状态$s$出发遵循策略$\pi$能获得的期望回报。它的数学表达为：
$$
V_{\pi}(s) = \mathbb{E}_\pi[G_t|S_t=s]
$$

### 动作价值函数（Action-value Function）
不同于MRP，在MDP中，由于动作的存在，我们额外定义一个动作价值函数。我们用$Q_{\pi}(s,a)$表示在MDP遵循策略$\pi$时，对当前状态$s$执行动作$a$得到的期望回报：
$$
Q_{\pi}(s,a)=\mathbb{E}_\pi[G_t|S_t=s,A_t=a]
$$

**Exercise 2 状态价值函数和动作价值函数之间的关系.**
1. 使用策略$\pi$，将状态价值函数$V_{\pi}(s)$用动作价值函数)$Q_{\pi}(s,a)$表示:\
$V_{\pi}(s) = \sum_{a \in A} \pi(a|s)Q_{\pi}(s,a)$
2. 在状态转移函数$p(s'|s,a)$下使用策略$\pi$，将动作价值函数$Q_{\pi}(s,a)$用状态价值函数$V_{\pi}(s)$表示:\
$Q_{\pi}(s,a) = \mathcal{r}(s,a) + \mathcal{\gamma}\sum_{s'\in S} p(s'|s,a)V_{\pi}(s')$


### 贝尔曼期望方程（Bellman Expectation Equation）
**Exercise 3 请分别写出状态价值函数和动作价值函数的贝尔曼期望方程.**\
$V_{\pi}(s) = \sum_{a \in A} \pi(a|s)(\mathcal{r}(s,a) + \mathcal{\gamma}\sum_{s'\in S} p(s'|s,a)V_{\pi}(s'))$\
$Q_{\pi}(s,a) =\mathcal{r}(s,a) + \mathcal{\gamma}\sum_{s'\in S} p(s'|s,a)\sum_{a \in A} \pi(a|s')Q_{\pi}(s',a)$


下图是一个MDP的简单例子，其中每个绿色的圆圈代表一个状态，一共有从$s_1$到$s_5$这5个状态。黑色实线箭头代表可以采取的动作，黄色的小圆圈代表动作，需要注意，并非在每个状态都能采取每个动作，例如在状态$s_1$，智能体只能采取“保持$s_1$”和“前往$s_2$”这两个动作，无法采取其他动作。
每个红色的数字代表在某个状态时采取某个动作能获得的奖励。蓝色虚线箭头代表采取动作后可能转移到的状态，箭头边上的数字是转移概率，如果没有数字表示转移概率为1。比如，在$s_2$下， 如果采取动作“前往$s_3$”，就能得到奖励-2，并且转移到$s_3$，如果采取“前往$s_1$”，就能得到奖励-1，并且转移到$s_1$。在$s_4$下，如果采取“概率前往”，就能得到奖励1，并且会以概率0.2, 0.4, 0.4转移到$s_2,s_3或s_4$。



![Image Name](https://cdn.kesci.com/upload/image/qged0vaxir.PNG?imageView2/0/w/400/h/400)
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; 图：马尔可夫决策过程的一个简单例子

接下来我们将图中马尔可夫决策过程用代码表示，并且定义两个策略。第一个策略是一个随机策略，即在每个状态的时候，会以同样的概率选取它可能采取的动作，例如在$s_1$会以0.5和0.5概率选取动作“保持$s_1$”和“前往$s_2$”。第二个策略是一个我们提前设定的一个策略。

In [4]:
# 状态集合
S = ["S1","S2","S3","S4","S5"]    
# 动作集合
A = ["保持S1","前往S1","前往S2","前往S3","前往S4","前往S5","概率前往"]
# 状态转移函数
P = {
    "S1-保持S1-S1":1.0, "S1-前往S2-S2":1.0,
    "S2-前往S1-S1":1.0, "S2-前往S3-S3":1.0,
    "S3-前往S4-S4":1.0, "S3-前往S5-S5":1.0,
    "S4-前往S5-S5":1.0, "S4-概率前往-S2":0.2,
    "S4-概率前往-S3":0.4, "S4-概率前往-S4":0.4,
}
# 奖励函数
R = {
    "S1-保持S1":-1, "S1-前往S2":0,
    "S2-前往S1":-1, "S2-前往S3":-2,
    "S3-前往S4":-2, "S3-前往S5":0,
    "S4-前往S5":10, "S4-概率前往":1,
}
# 折扣引子
gamma = 0.5

MDP = (S, A, P, R, gamma)

# 策略1，随机策略
Pi_1 = {                                         
    "S1-保持S1":0.5, "S1-前往S2":0.5,
    "S2-前往S1":0.5, "S2-前往S3":0.5,
    "S3-前往S4":0.5, "S3-前往S5":0.5,
    "S4-前往S5":0.5, "S4-概率前往":0.5,
}
# 策略2
Pi_2 = {                                        
    "S1-保持S1":0.6, "S1-前往S2":0.4,
    "S2-前往S1":0.3, "S2-前往S3":0.7,
    "S3-前往S4":0.5, "S3-前往S5":0.5,
    "S4-前往S5":0.1, "S4-概率前往":0.9,
}
# 把输入的两个字符串通过“-”连接，便于使用我们上述定义的P,R变量
def join(str1, str2): 
    return str1 + '-' + str2

接下来我们想要计算该MDP的价值函数。我们现在有的工具是MRP的解析解方法。于是，一个很自然的想法是：给定一个MDP和一个策略$\pi$，我们是否可以将其转化为一个MRP？答案是肯定的。我们可以将策略的动作选择进行边缘化（marginalization)，就可以得到没有动作的MRP了。具体的，对于某一个状态，我们计算根据策略所有动作的概率进行加权得到的奖励和，就可以认为是一个MRP在该状态下的奖励。即
$$
r'(s) = \sum_{a\in \mathcal{A}} \pi(a|s) r(s,a)
$$
同理，我们计算采取动作的概率与使$s$转移到$s'$的概率的乘积，再将这些乘积相加之和就是一个MRP的状态从$s$转移至$s'$的概率：
$$
p'(s'|s) = \sum_{a\in \mathcal{A}} \pi(a|s) p(s'|s,a)
$$
于是，我们构建得到了一个MRP:$<\mathcal{S}, p', r', \gamma>$。根据价值函数的定义，我们可以发现转化前的MDP的状态价值函数和转化后的MRP的价值函数是一样的。于是我们可以用MRP中计算价值函数的解析解来计算这个MDP中该策略的状态价值函数。

我们现在就用代码实现来用该方法计算用随机策略（也就是代码中的Pi_1）时的状态价值函数。为了简单起见，我们直接给出转化后的MRP的状态转移矩阵和奖励函数，同学们可以自行验证。

In [5]:
gamma = 0.5
# 转化后的MRP的状态转移矩阵
P_from_mdp_to_mrp = [
    [0.5, 0.5, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.5, 0.5],
    [0.0, 0.1, 0.2, 0.2, 0.5],
    [0.0, 0.0, 0.0, 0.0, 1.0],
]                           
P_from_mdp_to_mrp = np.array(P_from_mdp_to_mrp) 
R_from_mdp_to_mrp = [-0.5, -1.5, -1.0, 5.5, 0] 

V = compute(P_from_mdp_to_mrp, R_from_mdp_to_mrp, gamma, 5)
print("MDP中每个状态价值分别为\n", V)

MDP中每个状态价值分别为
 [[-1.22555411]
 [-1.67666232]
 [ 0.51890482]
 [ 6.0756193 ]
 [ 0.        ]]


在知道了状态价值函数$V$后，我们可以计算动作价值函数$Q$。例如（$s_4$，概率前往）的动作价值为2.152，根据以下式子可以计算得到：
$$
Q_{\pi}(s,a)=r(s,a)+\gamma \sum_{s' \in \mathcal{S}}p(s'|s,a)V_{\pi}(s') \\
2.152 = 1+0.5\times [0.2\times (-1.68)+0.4\times 0.52+0.4\times 6.08]
$$

这个MRP解析解的方法在状态动作集合比较大的时候不是很适用，那有没有其他的方法呢？在下一节课程中我们将介绍用动态规划的方法来计算得到价值函数。本节课程接下来部分我们介绍用蒙特卡洛方法来近似估计这个价值函数，用蒙特卡洛方法的好处在于我们不需要知道MDP的状态转移函数和奖励函数，但是它只能得到一个近似值，并且采样数越多越准确。

# 蒙特卡洛方法（Monte-Carlo methods）

蒙特卡洛方法也被称为统计模拟方法，是一种基于概率统计的数值计算方法。运用蒙特卡洛方法时，我们通常使用重复随机抽样，然后运用概率统计方法来从抽样结果中归纳出我们想求的目标的数值估计。一个简单的例子是用蒙特卡洛方法来计算圆的面积。例如在这个正方形内部，随机产生若干个点，细数落在圆中点的个数，图中圆的面积与正方形面积之比就等于圆中点的个数与正方形中点的个数之比。如果我们随机产生的点的个数越多，计算得到圆的面积就越接近于真实的圆的面积。
$$
\frac{圆的面积}{矩形的面积}=\frac{圆中点的个数}{矩形中点的个数}
$$


![Image Name](https://cdn.kesci.com/upload/image/qgad559l5m.png?imageView2/0/w/200/h/200)
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; 图：用蒙特卡洛方法估计圆的面积


我们现在介绍如何用蒙特卡洛方法来估计一个策略在一个马尔可夫决策过程中的状态价值函数。回忆一下一个状态的价值是它的期望回报，一个很直观的想法就是用策略在MDP上采样很多条序列，然后计算从这个状态出发的回报再求个平均值就可以了。也就是根据如下式子
$$
V_{\pi} = \mathbb{E}_{\pi}[G_t|S_t=s] ≃ \frac{1}{N}\sum_{i=1}^N G_t^{(i)}
$$
在一条序列中可能没出现这个状态，也可能只出现过一次这个状态，也可能出现过很多次这个状态。我们介绍的蒙特卡洛价值估计算法中，会对每一次该状态的出现都计算它的回报。还有一种选择是一条序列只计算一次回报，也就是这条序列第一次出现该状态时候计算后面的累计奖励，而后面再次出现该状态就被忽略了。假设我们现在用策略$\pi$从状态$s$开始采样序列，据此来计算状态价值。我们为每一个状态维护一个计数器和总回报。具体过程为
1. 使用策略$\pi$采样
$$
S_0^{(i)} \stackrel{A_0^{(i)}}{\longrightarrow} R_0^{(i)},S_1^{(i)} \stackrel{A_1^{(i)}}{\longrightarrow} R_1^{(i)},S_2^{(i)} ... R_{T-1}^{(i)}, S_T^{(i)}
$$
2. 对每一条序列中的每一时间步$t$的状态$s$进行以下操作
	* 更新状态$s$的计数器 $\ N(s)\leftarrow N(s)+1$
	* 更新状态$s$的总回报 $\ M(s)\leftarrow M(s)+G_t$

3. 每一个状态的价值被估计为回报的平均值 $\ V(s)=M(s)/N(s)$

根据大数定律，当$N(s)\rightarrow \infty$，我们有$V(s)\rightarrow V_{\pi}(s)$ 

计算回报的平均值，除了把所有的回报加起来处以次数，还有一种增量更新的方式。对于每个状态$s$和对应回报$G$，我们进行
* $N(s)\leftarrow N(s)+1$
* $V(s)\leftarrow V(s)+\frac{1}{N(s)}(G-V(s))$

可以这样做的原理如下

$$
\begin{align}
 V_k &=\frac{1}{k}\sum_{i=1}^{k} G_i \nonumber \\
 &= \frac{1}{k}(G_k + \sum_{i=1}^{k-1}G_i)  \nonumber \\
				&= \frac{1}{k}(G_k + (k-1)V_{k-1} + V_{k-1} - V_{k-1}) \nonumber \\
				&= \frac{1}{k}(G_k + kV_{k-1} - V_{k-1}) \nonumber \\
				&= V_{k-1} + \frac{1}{k}[G_k-V_{k-1}]   \nonumber
\end{align}
$$


接下来我们用代码定义一个采样函数，需要遵守状态转移矩阵和相应的策略，我们每次将（s,a,r,s_next）元组放入序列中，直到到达终止序列。然后我们通过该函数实现用随机策略在图中MDP实际采样几条序列。

In [6]:
# 采样函数，策略Pi，限制最长时间步timestep_max，总共采样序列数number
def sample(MDP, Pi, timestep_max, number):
    S, A, P, R, gamma = MDP
    episodes = []
    for _ in range(number):
        episode = []
        timestep = 0
        # 随机选择一个除了s5以外的状态s作为起点
        s = random.sample(S[:4], 1)[0]    
        # 当前状态为终止状态或者时间步太长时，一次采样结束
        while s != "S5" and timestep <= timestep_max:
            timestep += 1
            rand, temp = random.random(), 0 
            # 随机选取一个行动
            for a_opt in A:  
                temp += Pi.get(join(s, a_opt), 0)
                if temp > rand:
                    a = a_opt
                    r = R.get(join(s, a_opt), 0)
                    break
            rand, temp = random.random(), 0
            # 得到(状态s，采取行动a)之后的下一个状态s_next
            for s_opt in S:   
                temp += P.get(join(join(s, a), s_opt), 0)
                if temp > rand:
                    s_next = s_opt
                    break
            # 把（s,a,r,s_next）元组放入序列中
            episode.append((s, a, r, s_next))  
            # s_next变成当前状态，开始接下来的循环
            s = s_next                          
        episodes.append(episode)
    return episodes

# 采样5次，每个序列最长不超过1000步
episodes = sample(MDP, Pi_1, 1000, 5)
print('第一条序列\n', episodes[0])
print('第二条序列\n', episodes[1])
print('第五条序列\n', episodes[4])

第一条序列
 [('S1', '前往S2', 0, 'S2'), ('S2', '前往S3', -2, 'S3'), ('S3', '前往S5', 0, 'S5')]
第二条序列
 [('S2', '前往S1', -1, 'S1'), ('S1', '前往S2', 0, 'S2'), ('S2', '前往S1', -1, 'S1'), ('S1', '保持S1', -1, 'S1'), ('S1', '前往S2', 0, 'S2'), ('S2', '前往S1', -1, 'S1'), ('S1', '保持S1', -1, 'S1'), ('S1', '保持S1', -1, 'S1'), ('S1', '前往S2', 0, 'S2'), ('S2', '前往S3', -2, 'S3'), ('S3', '前往S4', -2, 'S4'), ('S4', '概率前往', 1, 'S3'), ('S3', '前往S5', 0, 'S5')]
第五条序列
 [('S1', '前往S2', 0, 'S2'), ('S2', '前往S1', -1, 'S1'), ('S1', '保持S1', -1, 'S1'), ('S1', '保持S1', -1, 'S1'), ('S1', '保持S1', -1, 'S1'), ('S1', '前往S2', 0, 'S2'), ('S2', '前往S1', -1, 'S1'), ('S1', '保持S1', -1, 'S1'), ('S1', '前往S2', 0, 'S2'), ('S2', '前往S1', -1, 'S1'), ('S1', '前往S2', 0, 'S2'), ('S2', '前往S1', -1, 'S1'), ('S1', '前往S2', 0, 'S2'), ('S2', '前往S1', -1, 'S1'), ('S1', '前往S2', 0, 'S2'), ('S2', '前往S3', -2, 'S3'), ('S3', '前往S4', -2, 'S4'), ('S4', '概率前往', 1, 'S3'), ('S3', '前往S4', -2, 'S4'), ('S4', '概率前往', 1, 'S4'), ('S4', '前往S5', 10, 'S5')]


In [7]:
# 对所有采样序列进行蒙特卡洛算法
def MC(episodes, V, N, gamma):
    for episode in episodes:
        G = 0
        for i in range(len(episode)-1, -1, -1):##一个序列从后往前计算
            (s, a, r, s_next) = episode[i]
            G = r + gamma * G  ## G_s = R + gamma * G_s_next 
            N[s] = N[s] + 1
            V[s] = V[s] + (G - V[s]) / N[s]


timestep_max = 10000
# 采样100000次，可以自行修改
episodes = sample(MDP, Pi_1, timestep_max, 100000)
gamma = 0.5
V = {"S1":0, "S2":0, "S3":0, "S4":0, "S5":0} 
N = {"S1":0, "S2":0, "S3":0, "S4":0, "S5":0}
MC(episodes, V, N, gamma)
print("使用蒙特卡洛法计算MDP状态价值为\n", V)

使用蒙特卡洛法计算MDP状态价值为
 {'S1': -1.2230453142164845, 'S2': -1.6764009028845834, 'S3': 0.5160752913967145, 'S4': 6.064663344981941, 'S5': 0}


可以看到用蒙特卡洛方法估计得到的状态价值和我们用MRP解析解得到的状态价值是很接近的。这当然得益于我们采样了比较多的序列，同学们可以尝试一下修改采样次数然后观察蒙特卡洛方法的结果。

# 占用度量（Occupancy Measure）

我们在之前提到，不同策略的价值函数是不一样的。但这是为什么呢？原因就在于对于同一个MDP，不同策略会访问到的状态分布是不同的。想象一下，在如图的MDP中，我们现在有一个策略，它的动作执行会使得智能体尽快到达终止状态$s_5$，于是当它处于状态$s_3$时，它不会采取“前往$s_4$”的动作，而只会以1的概率采取“前往$s_5$”的动作，所以它不会获得在$s_4$状态下采取“前往$s_5$”可以得到的很大的奖励10。可想而知，根据贝尔曼方程，这个策略在状态$s_3$的状态会比较小，究其原因是因为它没法到达状态$s_4$。所以我们需要理解不同策略会访问到不同分布的状态这件事实，它会影响到策略的价值函数。

首先我们定义MDP的初始状态分布为$\nu_0(s)$，在有些材料中，初始状态分布会被定义进MDP的组成元素中。我们用$P(S_t=s)$表示在$t$时刻状态为$s$的概率，所以我们有$P(S_0=s)=\nu_0(s)$，然后我们就可以定义一个策略的**状态访问分布（State Visitation Distribution）** 为
$$
\nu^{\pi}(s) = (1-\gamma) \sum_{t=0}^\infty \gamma ^t P_t^\pi(s)
$$
其中$1-\gamma$是用来使得概率加和为1的归一化因子。状态访问概率表示一个策略和MDP交互它会访问到的状态的分布，需要注意理论上计算该分布的公式中需要交互到无穷步之后，但实际情况中智能体和MDP的交互在一个序列中是有限的，不过我们仍然可以用以上公式来表达状态访问概率的思想。状态访问概率有如下性质
$$
\nu^{\pi}(s') = (1-\gamma) \nu_0(s') + \gamma \int  p(s'|s,a) \pi(a|s) \nu^{\pi}(s)  ds da
$$

此外，我们还可以定义**占用度量（Occupancy Measure）** 为
$$
\rho^{\pi}(s,a) = (1-\gamma)\sum_{t=0}^\infty \gamma ^t P_t^\pi(s) \pi(a|s)
$$
它表示（动作，状态）对被访问到的概率。两者之间存在如下关系
$$
\rho^{\pi}(s,a) = \nu^{\pi}(s) \pi(a|s) 
$$
我们有如下定理

- 定理1：和同一个MDP交互的两个策略$\pi_1$和$\pi_2$得到的占用度量$\rho^{\pi_1}$和$\rho^{\pi_2}$满足：$\rho^{\pi_1}=\rho^{\pi_2} \iff \ \pi_1 =\pi_2$

- 定理2：给定一占用度量$\rho$，可生成该占用度量的唯一策略是
$$
\pi_{\rho}=\frac{\rho(s,a)}{\sum_{a'}\rho(s,a')}
$$

接下来我们用代码来近似估计占用度量函数。这里我们采用近似估计，设置一个较大的episode length最大值，然后采样很多次，用$(s,a)$组出现的频率估计实际概率

In [8]:
# 限制最长时间步timestep_max，便于计算
def occupancy(episodes, s, a, timestep_max, gamma):
    rho = 0
    # 记录每个时间步t各被经历过几次
    total_times = [0 for _ in range(timestep_max)]
    # 记录(s_t,a_t)=(s,a)的次数
    occur_times = [0 for _ in range(timestep_max)]
    for episode in episodes:
        for i in range(len(episode)):
            (s_opt, a_opt, r, s_next) = episode[i]
            # 只要到达第t步，就+1
            total_times[i] += 1   
            # 如果(s_t,a_t)=(s,a)，就+1
            if s == s_opt and a == a_opt:     
                occur_times[i] += 1
    for i in range(timestep_max):
        if total_times[i]:
            # occur_times[i]/total_times[i]近似概率p(s_t=s,a_t=a)
            rho += gamma**i * occur_times[i]/total_times[i] 
    return (1 - gamma) * rho

In [9]:
gamma = 0.5
timestep_max = 10000

episodes_1 = sample(MDP, Pi_1, timestep_max, 100000)
episodes_2 = sample(MDP, Pi_2, timestep_max, 100000)
rho_1 = occupancy(episodes_1, "S4", "概率前往", timestep_max, gamma)
rho_2 = occupancy(episodes_2, "S4", "概率前往", timestep_max, gamma)
print(rho_1, rho_2)

0.11491433362750023 0.23589505934193272


我们可以发现不同策略对于同一个状态动作对的占用度量是不一样的

# 最优策略（Optimal Policy）
强化学习的目标通常是找到一个策略使得它从初始状态出发能获得最多的期望回报。我们首先定义策略之间的偏序关系为$\pi > \pi'$ 当且仅当 对于任意的状态$s$都有$V_\pi(s)\geq V_{\pi'}(s)$ 。于是在有限状态和动作集合的MDP中，至少存在一个策略比其他所有策略都好或者一样好，这个策略就是**最优策略**。尽管可能最优策略有很多个，我们都将其表示为
$$
\pi_{*} (s) 
$$

最优策略都有相同的状态价值函数，我们称之为**最优状态价值函数**，表示为：

$$
V_{*}(s) = \max_{\pi}V_{\pi}(s), \quad \forall s\in\mathcal{S}
$$

同理，我们定义**最优动作价值函数** :
$$
Q_{*}(s,a) = \max_{\pi}Q_{\pi}(s,a), \quad \forall s\in\mathcal{S}, a\in\mathcal{A}
$$

为了使得$Q_{\pi}(s,a)$最大，我们需要在当前的$(s,a)$之后都执行最优策略。于是我们得到了两者之间的关系：
$$
Q_{*}(s,a)=r(s,a)+\gamma \sum_{s' \in S}p(s'|s,a)V_{*}(s') 
$$
这和普通策略的状价值函数和动作价值函数之间的关系是一样的。另一方面，最优价值函数是选择此时最优动作状态最大的那一个动作时的价值：
$$
V_*(s) = \max_{a\in\mathcal{A}} Q_*(s,a)
$$


### 贝尔曼最优方程(Bellman Optimality Equation)
根据两者关系，我们可以得到**贝尔曼最优方程(Bellman Optimality Equation)**：

$$V^{*}(s)=\max _{a \in \mathcal{A}}\left\{r(s, a)+\gamma \sum_{s^{\prime} \in \mathcal{S}} p\left(s^{\prime} \mid s, a\right) V^{*}\left(s^{\prime}\right)\right\}$$

$$Q^{*}(s, a)=r(s, a)+\gamma \sum_{s^{\prime} \in \mathcal{S}} p\left(s^{\prime} \mid s, a\right) \max _{a^{\prime} \in \mathcal{A}} Q^{*}\left(s^{\prime}, a^{\prime}\right)$$


# 基于动态规划的强化学习算法

动态规划（Dynamic Programming）是程序设计算法中非常重要的内容，能够高效解决一些经典问题，例如背包问题和最短路径规划。动态规划的基本思想是将待求解问题分解成若干个子问题，先求解子问题，然后从这些子问题的解得到原问题的解。在动态规划中，我们会保存已解决的子问题的答案，而在求解目标问题过程中，如果需要这些子问题答案时，就可以直接利用，避免重复计算。在本节课程内容中，我们就学习如何用动态规划的思想来求解在有限马尔可夫决策过程中的最优策略。

基于动态规划的强化学习算法主要有两种：一是策略迭代（Policy Iteration），二是价值迭代(Value Iteration)。其中，策略迭代有两部分组成：策略评估（Policy Evaluation）和策略提升（Policy Improvement）。具体来说，策略迭代中的策略评估使用贝尔曼期望方程来得到一个策略的状态价值函数，这是一个动态规划的过程。而价值迭代直接使用贝尔曼最优方程来进行动态规划，得到最终的最优状态价值。

不同于我们之前介绍的蒙特卡洛算法和之后将要介绍的时序差分算法，基于动态规划的两大强化学习算法要求我们能事先知道环境的状态转移函数和奖励函数。在这样一个白盒环境中，我们不需要通过智能体和环境的大量交互来学习，可以直接用动态规划求解状态价值。但是，现实中白盒环境很少，这也是动态规划算法的局限所在，我们无法将其运用到很多实际场景中。其次，我们介绍的策略迭代和价值迭代通常只适用于有限马尔可夫决策过程中，即状态空间和动作空间是有限的。


# Cliff Walking环境介绍

本节课程中，我们将使用策略迭代和价值迭代来求解Cliff Walking这个环境中的最优策略。接下来我们将简单介绍该环境，示意图如下。

![Image Name](https://cdn.kesci.com/upload/image/qgsh87rffu.png?imageView2/0/w/640/h/640)
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; 图：Cliff Walking环境示意图

Cliff Walking是一个非常经典的强化学习环境，它要求一个智能体从起点出发，避开悬崖行走，最终到达目标位置。如图所示，有一个$4*12$ 的网格世界，每一个网格是一个状态，起点是左下角的状态，目标是右下角的状态。智能体在每一个状态都可以采取4种动作：上,下,左,右，如果采取动作后触碰到边界墙壁则状态不发生改变，否则就会相应到达下一个状态。其中有一段悬崖，智能体到达目标状态或掉入悬崖都会结束并回到起点，也就是说它们是终止状态。每走一步的奖励是-1，掉入悬崖的奖励是-100。

接下来一起来看一看Cliff Walk环境的代码吧。

In [10]:
class CliffWalking(object):
    """
    Cliff Walking
    """
    def __init__(self, width=12, height=4):
        # 定义环境的宽，高，动作空间
        self.nrow = width
        self.ncol = height
        self.action_space = 4
        # 定义起点，坐标轴原点在网格世界的左上角
        self.x = self.ncol - 1
        self.y = 0
        # 状态转移矩阵P[state][action] = [(p, next_state, reward, done)]
        self.P = self.createP()

    def createP(self):
        P = [[[] for j in range(self.action_space)]
             for i in range(self.ncol * self.nrow)]
        # 4 种动作, 0:上, 1:下, 2:左, 3:右
        change = [[-1, 0], [1, 0], [0, -1], [0, 1]]
        for i in range(self.ncol):
            for j in range(self.nrow):
                for a in range(self.action_space):
                    if i == self.ncol - 1 and j > 0:  # 位置在悬崖或者终点
                        if j != self.nrow - 1:  # 悬崖
                            P[i * self.nrow +
                                j][a] = [(1, i * self.nrow + j, -100, True)]
                        else:  # 终点
                            P[i * self.nrow +
                                j][a] = [(1, i * self.nrow + j, 0, True)]
                        continue
                    # 其他位置
                    next_c = min(self.ncol - 1, max(0, i + change[a][0]))
                    next_r = min(self.nrow - 1, max(0, j + change[a][1]))
                    next_state = next_c * self.nrow + next_r
                    reward = -1
                    done = False
                    if next_c == self.ncol - 1:  
                        if next_r > 0:  # 下一个位置在悬崖或者终点
                            done = True
                            if next_r != self.nrow - 1:  # 悬崖
                                reward = -100
                    P[i * self.nrow + j][a] = [(1, next_state, reward, done)]
        return P

    def reset(self):
        # 坐标重置到起点
        self.x = self.ncol - 1
        self.y = 0

# 策略迭代

策略迭代是策略评估和策略提升这两个过程的不断循环交替，直至最后得到最优策略。我们接下来分别进行详细介绍。

### 策略评估

策略评估这一过程用来计算一个策略的状态价值函数。回顾一下之前学习的贝尔曼期望方程，见**Exercise 3**。


我们看到，当我们知道奖励函数和状态转移函数时，我们可以根据下一个状态的价值来计算当前状态的价值。于是，根据动态规划的思想，我们可以将计算下一个可能状态的价值当成一个子问题，计算当前状态的价值看成当前问题。在得知子问题的解后，我们就可以求解当前问题。更一般的，我们考虑所有的状态，就变成了我们用上一轮的状态价值函数来计算当前这一轮的状态价值函数$V(s)$。

**Exercise 4 写出状态价值函数的更新公式。**\
$$V_{k+1}^{\pi}(s) \leftarrow \sum_{a \in A} \pi(a|s)(\mathcal{r}(s,a) + \mathcal{\gamma}\sum_{s'\in S} p(s'|s,a)V_{k}^{\pi}(s'))$$


我们可以选定任意初始值$V_0$。根据贝尔曼方程，我们得知$V_{k}=V_\pi$是以上更新公式的一个不动点（fixed point）。事实上，我们可以证明（见扩展阅读）当$k\rightarrow \infty$时，序列$\{ V_k \}$会收敛到$V_\pi$。所以我们可以据此来计算得到一个策略的状态价值函数。在实际实现过程中，在某一轮如果$\max_{s\in\mathcal{S}}|V_{k+1}(s)-V_k(s)|$的值非常小时，我们会提前结束策略评估。这样做可以提升效率，并且得到的价值函数也非常接近真实的价值函数。


### 策略提升

在用策略评估计算得到当前策略的状态价值函数之后，我们可以据此来改进该策略，这个步骤称为策略提升。假设此时对于策略$\pi$，我们已经知道其状态价值函数$V_\pi$，也就是我们知道了从某一个状态$s$出发一直根据策略$\pi$最终得到的期望回报。但是我们要如何改变策略来获得在状态$s$下更高的期望回报呢？假设我们在状态$s$下采取动作$a$然后之后的动作依旧遵循策略$\pi$，此时得到的期望回报其实就是动作价值$Q_\pi(s,a)$。如果我们有$Q_\pi(s,a)>V_\pi(s)$，则说明在状态$s$下采取动作$a$会比原来的策略$\pi(a|s)$得到更高的期望回报。以上只是针对一个状态。现在假设存在一个确定性策略$\pi'$，在任意一个状态$s$下，都满足
$$
Q_\pi(s, \pi'(s)) \geq V_\pi(s)，
$$
于是我们有在任意状态$s$下，
$$
V_{\pi'}(s) \geq V_{\pi}(s)。
$$
这便是策略提升定理（Policy Improvement Theorem）。于是我们可以直接贪心地在每一个状态选择动作价值最大的动作，也就是
$$
\pi'(s) = \arg\max_a Q_\pi(s,a) = \arg\max_a r(s,a)+\gamma \sum_{s'} p(s'|s,a)V_\pi(s')
$$
我们发现构造的贪心策略$\pi'$满足策略提升定理的条件，所以策略$\pi'$能够比策略$\pi$更好或者与其一样好。这个根据贪心法选取动作得到新的策略的过程称为策略提升。当构建得到的策略$\pi'$和之前的策略$\pi$一样好时，即$V_\pi=V_{\pi'}$，此时对于任意的状态$s$都有
$$
V_{\pi'}(s) = \max_a r(s,a)+\gamma \sum_{s'} p(s'|s,a)V_{\pi'}(s')。
$$
这和贝尔曼最优方程一致，所以此时$\pi$和$\pi'$就是最优策略。

**Exercise 5 证明策略提升定理。**\
Proof:

设当前策略为$\pi(s)$，状态价值函数为$V_{\pi}(s)$，而下一个确定性策略为$\pi'$\
已知：对于任意状态$s \in S$，满足：
$$
Q_\pi(s, \pi'(s)) \geq V_\pi(s)
$$
对于任意状态$s\in S$，在当前的策略下的价值状态函数为：
$$
V_{\pi}(s) = \sum_{a \in A} \pi(a|s)(\mathcal{r}(s,a) + \mathcal{\gamma}\sum_{s'\in S} p(s'|s,a)V_{\pi}(s'))
$$
则当使用新的确定性的策略$\pi'$后，第一次迭代的状态价值函数$V^{\pi'}_1(s)$为：
$$
V^{\pi'}_1(s) = \sum_{a \in A} \pi'(a|s)Q_{\pi}(s, a) = Q_\pi(s, \pi'(s))
$$
设$V_{\pi}(s)$ 为$V^{\pi'}_0(s)$，则$V^{\pi'}_1(s) \geq V^{\pi'}_0(s)$。\
对于第i轮迭代，若满足对于任意$s$，使得$V^{\pi'}_i(s) \geq V^{\pi'}_{i-1}(s)$。(i=1,2,3....)\
则对于第i+1轮迭代：
$$
V^{\pi'}_{i+1}(s) = \sum_{a \in A} \pi'(a|s)(\mathcal{r}(s,a) + \mathcal{\gamma}\sum_{s'\in S} p(s'|s,a)V_{i}^{\pi'}(s'))=\mathcal{r}(s,\pi'(s)) + \mathcal{\gamma}\sum_{s'\in S} p(s'|s,\pi'(s))V_{i}^{\pi'}(s')
$$
同理，对于第i轮迭代：
$$
V^{\pi'}_{i}(s) = \sum_{a \in A} \pi'(a|s)(\mathcal{r}(s,a) + \mathcal{\gamma}\sum_{s'\in S} p(s'|s,a)V_{i-1}^{\pi'}(s'))=\mathcal{r}(s,\pi'(s)) + \mathcal{\gamma}\sum_{s'\in S} p(s'|s,\pi'(s))V_{i-1}^{\pi'}(s')
$$
则：
$$
V^{\pi'}_{i+1}(s)-V^{\pi'}_{i}(s) = \mathcal{\gamma}\sum_{s'\in S} p(s'|s,\pi'(s))(V_{i}^{\pi'}(s')-V_{i-1}^{\pi'}(s'))
$$
由于对于任意$s$，使得$V^{\pi'}_i(s) \geq V^{\pi'}_{i-1}(s)$。(i=1,2,3....)，则$V^{\pi'}_{i+1}(s)-V^{\pi'}_{i}(s) \geq 0$\
当i趋于无穷时，$V^{\pi'}_i(s)$收敛到$V_{\pi'}(s)$。\
则$V_{\pi'}(s)\geq ...\geq V^{\pi'}_i(s) \geq V^{\pi'}_{i-1}(s) \geq...\geq V^{\pi'}_0(s) =  V_{\pi}(s)$，
证毕！

### 策略迭代算法
总体来说，策略迭代算法为我们对当前的策略进性策略评估，得到其状态价值函数，然后用该状态价值函数根据策略提升来得到一个更好的新策略，然后再继续评估新策略再提升策略，以此往复，直至最后收敛到最优策略：
$$
\pi_0 \stackrel{策略评估}{\longrightarrow} V_{\pi_0} \stackrel{策略提升}{\longrightarrow} \pi_1  \stackrel{策略评估}{\longrightarrow} V_{\pi_1} \stackrel{策略提升}{\longrightarrow} \pi_2  \stackrel{策略评估}{\longrightarrow}    ...\stackrel{策略提升}{\longrightarrow}  \pi_*
$$
结合策略评估和策略提升，我们得到以下策略迭代算法。
1. **初始化**

    对于所有状态 $s \in \mathcal{S}$，任意初始化策略$\pi(s)$和价值函数$V(s)$

2. **策略评估**
不断进行如下循环:

    > $\Delta \leftarrow 0$  
    > 对于每一个状态 $s \in \mathcal{S}$:  
    >> $v \leftarrow V(s)$  
    >> $V(s) \leftarrow r(s, \pi(s)) + \gamma \sum_{s'}  p(s' | s, \pi(s)) V(s')$  
    >> $\Delta \leftarrow \max(\Delta, |v-V(s)|)$
    
    直到 $\Delta < \theta$，结束循环

3. **策略提升**  
	
	布尔变量flag初始化为TRUE
    对于每一个状态 $s \in \mathcal{S}$:
    
    > $a \leftarrow \pi(s)$  
    > $\pi(s) \leftarrow \arg \max_a r(s,a) + \gamma \sum_{s'}p(s' | s, a) V(s')$  
    > 若 $a \ne \pi(s)$, 则 flag $\leftarrow$ FALSE
    
    若 flag为TRUE, 则停止算法并返回 $V$ 和 $\pi$; 否则转到 2.策略评估
		
我们现在来看一下策略迭代的代码。

In [11]:
class AgentPI(object):
    """策略迭代"""
    def __init__(self, env, action_meaning, dead=[], end=[], theta=0.001, gamma=0.9):
        self.env = env
        # 两种终止点，只用于打印策略
        self.dead = dead
        self.end = end

        self.width = env.nrow
        self.height = env.ncol
        self.actions = 4
        self.action_meaning = action_meaning

        self.v = [0] * self.width * self.height  # 初始化价值为0
        self.pi = [[0.25, 0.25, 0.25, 0.25] for i in range(self.width * self.height)]  # 初始化为均匀随机策略

        self.theta = theta  # 价值收敛阈值
        self.gamma = gamma  # 折扣银子

    def update(self):
        # 更新状态价值
        d = 0
        newv = [0] * self.width * self.height
        for s in range(self.height * self.width):
            ########################################
            ## Programming 2: 状态价值函数更新
            ########################################
            # 记录最大更新量，判断是否收敛
            pi_s = self.pi[s]
            p_as = self.env.P[s]
            for a in range(self.actions):
                tmp = p_as[a][0]
                newv[s] += tmp[2] * pi_s[a]
                if not tmp[3]:
                    newv[s] += pi_s[a] * self.gamma*self.v[tmp[1]]
            d = max(d, abs(newv[s] - self.v[s]))
        self.v = newv
        return d

    def policy_evaluation(self):
        # 价值更新直到收敛
        cnt = 1
        while (self.update() > self.theta):
            cnt += 1
        print("----evaluation done after %d iterations----" % cnt)

    def policy_improvement(self):
        # 策略更新直到稳定
        stable = True
        for s in range(self.height * self.width):
            new_a = [0,0,0,0]
            p_sa = self.env.P[s]
            q = [0,0,0,0]
            max_q = -100000000
            num = 0
            for a in range(self.actions):
                tmp = p_sa[a][0]
                q[a] = tmp[2] 
                if not tmp[3]:
                    q[a] += self.gamma * self.v[tmp[1]]
                q[a] = round(q[a],14)
                    
                if max_q < q[a]:
                    max_q = q[a]
                    num = 1
                            
                elif max_q == q[a]:
                    num += 1
                    
            for a in range(self.actions):
                if max_q == q[a]:
                    new_a[a] = 1/num
        
            if self.pi[s] != new_a:
                stable = False
                self.pi[s] = new_a
            ########################################
            ## Programming 3: 策略提升，注意让相同的动作价值均分概率
            # 判断是否收敛
            # if self.pi[s] != '新的策略':  
            #     stable = False
            #     self.pi[s] = '新的策略'
            ########################################
        return stable

    def policy_iteration(self):
        # 重复策略评估和策略提升直到策略稳定
        cnt = 0
        while True:
            print("---- %d iterations----" % cnt)
            self.policy_evaluation()
            stable = self.policy_improvement()
            cnt += 1

            if stable:
                self.print_value()
                self.print_policy()
                break

    def print_value(self):
        # 打印状态价值
        print("value:")
        for i in range(self.height):
            for j in range(self.width):
                print('%.3f' % self.v[i * self.width + j], end=' ')
            print()

    def print_policy(self):
        # 打印策略
        print("policy:")
        for i in range(self.height):
            for j in range(self.width):
                if (i * self.width + j) in self.dead:
                    print('****', end=' ')
                elif (i * self.width + j) in self.end:
                    print('EEEE', end=' ')
                else:
                    a = self.pi[i * self.width + j]
                    pi_str = ''
                    for k in range(4):
                        pi_str += self.action_meaning[k] if a[k] > 0 else 'o'
                    print(pi_str, end=' ')
            print()

**Exercise 6** 上文的策略迭代算法中，我们是用来价值函数$𝑉(𝑠)$来评估策略，那么如何使用动作价值函数$Q(s,a)$实现策略迭代算法？请类比上文的算法描述给出完整算法（伪代码）。

1. **初始化**

    对于所有状态 $s \in \mathcal{S}$，任意初始化策略$\pi(s)$和动作价值函数$Q(s,a)$
2. **策略评估**
不断进行如下循环:

    > $\Delta \leftarrow 0$  
    > 对于每一个状态 $s \in \mathcal{S}$:  
    >> $q \leftarrow Q(s,a)$  
    >> $Q(s,\pi(s)) \leftarrow \mathcal{r}(s,\pi(s)) + \mathcal{\gamma}\sum_{s'\in S} p(s'|s,a)Q(s,\pi(s'))$  
    >> $\Delta \leftarrow \max(\Delta, |p-Q(s,\pi(s)|)$
    
    直到 $\Delta < \theta$，结束循环

3. **策略提升**  
	
	布尔变量flag初始化为TRUE
    对于每一个状态 $s \in \mathcal{S}$:
    
    > $a \leftarrow \pi(s)$  
    > $\pi(s) \leftarrow \arg \max_a Q(s,a)$  
    > 若 $a \ne \pi(s)$, 则 flag $\leftarrow$ FALSE
    
    若 flag为TRUE, 则停止算法并返回 $Q$ 和 $\pi$; 否则转到 2.策略评估

现在我们已经写好了环境代码和策略迭代代码，看一下运行效果。其中打印出来的策略表示在每个状态它采取怎样的动作，例如"^o<o"表示等概率采取向左和向上两种动作，"ooo>"表示在当前状态只采取向右动作。

In [12]:
env = CliffWalking()
action_meaning = ['^', 'v', '<', '>']
agent = AgentPI(env, action_meaning, list(range(37, 47)), [47])
agent.policy_iteration()

---- 0 iterations----
----evaluation done after 60 iterations----
---- 1 iterations----
----evaluation done after 72 iterations----
---- 2 iterations----
----evaluation done after 44 iterations----
---- 3 iterations----
----evaluation done after 12 iterations----
---- 4 iterations----
----evaluation done after 1 iterations----
value:
-7.712 -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 
-7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 
-7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 -1.000 
-7.458 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 0.000 
policy:
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ovoo 
^ooo **** **** **** **** **** **** **** **** **** **** EEEE 


正确输出应为：

```plain
---- 0 iterations----
----evaluation done after 60 iterations----
---- 1 iterations----
----evaluation done after 72 iterations----
---- 2 iterations----
----evaluation done after 44 iterations----
---- 3 iterations----
----evaluation done after 12 iterations----
---- 4 iterations----
----evaluation done after 1 iterations----
value:
-7.712 -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 
-7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 
-7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 -1.000 
-7.458 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 0.000 
policy:
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ovoo 
^ooo **** **** **** **** **** **** **** **** **** **** EEEE
```

经过五轮策略评估和策略提升的循环迭代，策略收敛了。

# 价值迭代

策略迭代中的策略评估需要很多轮才能收敛得到某一策略的状态函数，这需要很大的计算量。我们是否必须要完全等到策略评估完成？试想一下，可能出现这样的情况：虽然状态价值函数还没收敛，但是不论接下来怎么更新状态价值，策略提升得到的可能是同一个策略。更特殊一点，如果我们只在策略评估中进行一轮价值的更新，然后直接根据更新后的价值进行策略提升，这样是否可以呢？其实这就是价值迭代，我们可以认为它是一种策略评估中只进行了一次更新的策略迭代。需要注意的是，价值迭代中不存在一个显式的策略。


确切来说，价值迭代可以看成一种动态规划过程，它利用的是贝尔曼最优方程:

$$V_{*}(s)=\max _{a \in \mathcal{A}}\left\{r(s, a)+\gamma \sum_{s^{\prime} \in \mathcal{S}} p\left(s^{\prime} \mid s, a\right) V_{*}\left(s^{\prime}\right)\right\}$$

**Exercise 7 写出价值迭代状态价值函数的更新方程。**
$$V^{*}_{i+1}(s) \leftarrow \max _{a \in \mathcal{A}}\left\{r(s, a)+\gamma \sum_{s^{\prime} \in \mathcal{S}} p\left(s^{\prime} \mid s, a\right) V^{*}_{i}\left(s^{\prime}\right)\right\}$$

价值迭代便是按照以上更新方式进行。等到$V_{k+1}$和$V_{k}$一样时，它就是贝尔曼最优方程的不动点，此时对应着最优状态价值函数$V_*$。然后我们从中恢复出最优策略即可：$\pi(s) = \arg \max_{a} r(s,a) + \gamma  \sum_{s'}p(s' | s, a) V_{k+1}(s')$。

### 价值迭代算法

 对所有状态 $s \in \mathcal{S}$，任意初始化 $V(s)$

不断进行如下循环:

>  $\Delta \leftarrow 0$  
>  对于每一个状态 $s \in \mathcal{S}$:  
>> $v \leftarrow V(s)$  
>> $V(s) \leftarrow \max_{a} r(s,a) + \gamma \sum_{s'}p(s' | s, a)V(s')$  
>> $\Delta \leftarrow \max(\Delta, |v-V(s)|)$  

直到 $\Delta < \theta$，结束循环

返回一个确定性策略 $\pi(s) = \arg \max_{a} r(s,a) + \gamma  \sum_{s'}p(s' | s, a) V(s')$

我们现在来实现价值迭代的代码，并将其用在Cliff Walking环境中

In [13]:
class AgentVI(object):
    """价值迭代"""
    def __init__(self, env, action_meaning, dead=[], end=[], theta=0.001, gamma=0.9):
        self.env = env
        # 两种终止点
        self.dead = dead
        self.end = end

        self.width = env.nrow
        self.height = env.ncol
        self.actions = 4
        self.action_meaning = action_meaning

        self.v = [0] * self.width * self.height  # 初始化价值为0
        self.pi = [[0.25, 0.25, 0.25, 0.25] for i in range(self.width * self.height)]

        self.theta = theta  # 价值收敛阈值
        self.gamma = gamma  # 衰减因子

    def update(self):
        # 更新状态价值
        d = 0
        newv = [0] * self.width * self.height
        for s in range(self.height * self.width):
            ########################################
            ## Programming 4：状态价值函数更新
            ########################################
            # 记录最大更新量，判断是否收敛
            p_as = self.env.P[s]
            q_max = -1000000
            for a in range(self.actions):
                tmp = p_as[a][0]
                q = tmp[2]
                if not tmp[3]:
                    q += self.gamma * self.v[tmp[1]]
                if q > q_max:
                    q_max = q
            newv[s] = q_max
            d = max(d, abs(newv[s] - self.v[s]))
        self.v = newv
        return d

    def value_iteration(self):
        # 价值迭代直到收敛
        cnt = 0
        while (self.update() > self.theta):
            cnt += 1
        print("----total %d iterations----" % cnt)
        self.print_value()
        self.get_policy()
        self.print_policy()

    def get_policy(self):
        # 根据价值计算策略
        for s in range(self.height * self.width):
            
            ########################################
            ## Programming 5：得到策略函数
            # self.pi[s] = '新的策略'
            ########################################
            p_as = self.env.P[s]
            new_pi = np.array([0,0,0,0])
            max_q = -1000000
            q = [0,0,0,0]
            for a in range(self.actions):
                tmp = p_as[a][0]
                q[a] += tmp[2]
                if not tmp[3]:
                    q[a] += self.gamma * self.v[tmp[1]]
                if max_q < q[a]:
                    max_q = q[a]
            for a in range(self.actions):
                if q[a] == max_q:
                    new_pi[a] = 1
            new_pi = list(new_pi / new_pi.sum())
            self.pi[s] = new_pi
    
    def print_value(self):
        # 打印价值
        print("value:")
        for i in range(self.height):
            for j in range(self.width):
                print('%.3f' % self.v[i * self.width + j], end=' ')
            print()
    
    def print_policy(self):
        # 打印策略
        print("policy:")
        for i in range(self.height):
            for j in range(self.width):
                if (i * self.width + j) in self.dead:
                    print('****', end=' ')
                elif (i * self.width + j) in self.end:
                    print('EEEE', end=' ')
                else:
                    a = self.pi[i * self.width + j]
                    pi_str = ''
                    for k in range(4):
                        pi_str += self.action_meaning[k] if a[k] > 0 else 'o'
                    print(pi_str, end=' ')
            print()
            
env = CliffWalking()
action_meaning = ['^', 'v', '<', '>']
agent = AgentVI(env, action_meaning, list(range(37, 47)), [47])
agent.value_iteration()

----total 14 iterations----
value:
-7.712 -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 
-7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 
-7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 -1.000 
-7.458 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 0.000 
policy:
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ovoo 
^ooo **** **** **** **** **** **** **** **** **** **** EEEE 


正确输出应为：
```plain
----total 14 iterations----
value:
-7.712 -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 
-7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 
-7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 -1.000 
-7.458 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 -100.000 0.000 
policy:
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ovoo 
^ooo **** **** **** **** **** **** **** **** **** **** EEEE
```

# 扩展阅读：收敛性证明

### 策略迭代
策略迭代的过程如下：
$$
\pi_0 \stackrel{策略评估}{\longrightarrow} V_{\pi_0} \stackrel{策略提升}{\longrightarrow} \pi_1  \stackrel{策略评估}{\longrightarrow} V_{\pi_1} \stackrel{策略提升}{\longrightarrow} \pi_2  \stackrel{策略评估}{\longrightarrow}    ...\stackrel{策略提升}{\longrightarrow}  \pi_*
$$
并且根据策略提升定理，我们知道更新后的策略的价值函数满足单调性，即$V_{\pi_{k+1}} \geq V_{\pi_k}$。所以只要所有可能的策略个数是有限的，那么策略迭代就能收敛到最优策略。假设MDP的状态空间大小为$|\mathcal{S}|$，动作空间大小为$|\mathcal{A}|$，此时所有可能的策略个数为$|\mathcal{A}|^{|\mathcal{S}|}$，是有限个。所以策略迭代能够在有限步能找到其中的最优策略。


### 价值迭代
价值迭代的更新公式为
$$
V_{k+1}(s) = \max_{a \in \mathcal{A}} \{r(s,a) + \gamma \sum_{s' \in \mathcal{S}}p(s'|s,a) V_k(s')\}
$$
我们将其定义为一个**贝尔曼最优算子**$\mathcal{T}$:
$$
V_{k+1}(s) = \mathcal{T}V_k(s) = \max_{a \in \mathcal{A}} \{r(s,a) + \gamma \sum_{s' \in \mathcal{S}}p(s'|s,a) V_k(s')\}
$$
然后我们引入**压缩算子**（Contraction Operator）的定义：若$O$是一个算子，如果满足$||OV-OV' ||_q \leq ||V-V'||_q$条件，则我们称$O$是一个压缩算子。其中$||x||_q$表示$x$的$L_q$范数，包括我们将会用到的无穷范数$||x||_\infty= \max_i |x_i|$
我们接下来证明当$\gamma<1$时，贝尔曼最优算子$\mathcal{T}$是一个$\gamma$-压缩算子。
$$
\begin{align}
 || \mathcal{T}V - \mathcal{T}V' ||_\infty & = \max_{s \in \mathcal{S}} | \max_{a\in \mathcal{A}} \{r(s,a) + \gamma \sum_{s' \in \mathcal{S}}p(s'|s,a) V(s')\} - \max_{a'\in \mathcal{A}} \{r(s,a') + \gamma \sum_{s' \in \mathcal{S}}p(s'|s,a') V'(s')\}  | \nonumber \\
 & \leq \max_{s,a} |  r(s,a) + \gamma \sum_{s' \in \mathcal{S}}p(s'|s,a) V(s') - r(s,a) - \gamma \sum_{s' \in \mathcal{S}}p(s'|s,a) V'(s') | \nonumber \\
				& = \gamma \max_{s, a} |  \sum_{s' \in \mathcal{S}}p(s'|s,a) (V(s')-V'(s'))  | \nonumber \\
				& \leq \gamma \max_{s, a} |  \sum_{s' \in \mathcal{S}}p(s'|s,a) | \max_{s'} | V(s')-V'(s')  | \nonumber \\
				& = \gamma ||V-V'||_\infty  \nonumber
\end{align}
$$
于是我们有
$$
|| V_{k+1}-V_*||_\infty = ||\mathcal{T}V_{k}- \mathcal{T} V_*||_\infty \leq \gamma || V_k-V_* ||_\infty \leq \cdots \leq \gamma^{k+1}||V_0-V_*||_\infty
$$
这意味着，在$\gamma \leq 1$的情况下，当$k$越来越大，即迭代次数越来越多的时候，$V_k$会越来越接近$V_*$，即$\text{lim}_{k\rightarrow \infty} V_k = V_*$。至此，价值迭代的收敛性得到证明。